In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
tokens = tokenizer.encode('I absolutely adore this. I love it.', return_tensors = 'pt')

tokens

tensor([[  101,   151, 35925, 10563, 39098, 10247, 10372,   119,   151, 11157,
         10197,   119,   102]])

In [4]:
tokens[0]

tensor([  101,   151, 35925, 10563, 39098, 10247, 10372,   119,   151, 11157,
        10197,   119,   102])

In [5]:
tokenizer.decode(tokens[0])

'[CLS] i absolutely adore this. i love it. [SEP]'

In [6]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.3912, -2.9581, -1.3648,  1.2215,  4.3913]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [7]:
result.logits

tensor([[-2.3912, -2.9581, -1.3648,  1.2215,  4.3913]],
       grad_fn=<AddmmBackward0>)

In [8]:
int(torch.argmax(result.logits))+1

5

In [9]:
URL = 'https://www.yelp.com/biz/social-brew-cafe-pyrmont?start='
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}  # Example user-agent string
number_of_pages = 14
all_reviews = []

In [10]:
for page_num in range(number_of_pages):
  start_param = page_num * 10
  url = f"{URL}{start_param}"
  r = requests.get(url)
  soup = BeautifulSoup(r.text,'html.parser')
  regex = re.compile('.*comment.*')
  results = soup.find_all('p', {'class': regex } )
  reviews = [result.text for result in results]
  all_reviews.extend(reviews)

In [11]:
import numpy as np
import pandas as pd

In [12]:
df = pd.DataFrame(np.array(all_reviews), columns = ['reviews'])
df

,reviews
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,"Great service, lovely location, and really ama..."
3,Great place with delicious food and friendly s...
4,Some of the best Milkshakes me and my daughter...
...,...
139,Dropped by here for a bite since it started to...
140,The breakfast options we had were delicious. ...
141,Conveniently located opposite the convention c...
142,I came here on a Sunday at 11am and unfortunat...


In [13]:
df['reviews'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [14]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors = 'pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [15]:
sentiment_score(df['reviews'].iloc[1])

4

In [16]:

df['sentiment'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [17]:
df['sentiment']

0      4
1      4
2      5
3      5
4      5
      ..
139    4
140    4
141    4
142    2
143    4
Name: sentiment, Length: 144, dtype: int64

In [18]:
df['sentiment'].value_counts()

5    92
4    32
3    10
2     5
1     5
Name: sentiment, dtype: int64